In [497]:
import pandas as pd
import numpy as np
import warnings

from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import precision_score

from sys import platform

if platform == "win32":
    path = 'C:/Users/olive/GitHub/f1-analytics/'
elif platform == "darwin":
    path = '~/Documents/GitHub/f1-analytics/'
    # path = '/Users/oliverjcarter/Documents/GitHub/f1-analytics/'

warnings.filterwarnings("ignore", category=RuntimeWarning) 
pd.options.mode.chained_assignment = None  # default='warn'

%matplotlib inline

In [498]:
data = pd.read_csv(path+'data/ml_input.csv')
processed = pd.read_csv(path+'data/processed.csv')
parameters = pd.read_csv(path+'parameters/rf_regressor.csv')

In [499]:
### Season to test results

N = 2021

In [500]:
params_to_drop = ['season', 'round', 'driver', 'constructor', 'circuit_id', 'podium', 'driver_points_from']

In [501]:
df = data.copy()

train = df[df.season < N]

scaler = StandardScaler()

X_train = train.drop(params_to_drop, axis=1)
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)

y_train = np.asarray(train.driver_points_from.values)

In [502]:
def get_predictions(X_train, y_train, model):
    score = 0
    points = []
    winners = []

    for circuit in df[df.season == N]['round'].unique():

        test = df[(df.season == N) & (df['round'] == circuit)]
        X_test = test.drop(params_to_drop, axis=1)
        y_test = test.driver_points_from

        #scaling
        X_test = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

        # make predictions
        prediction_df = pd.DataFrame(model.predict(X_test), columns=['predicted_points'])
        prediction_df['actual_points'] = y_test.reset_index(drop=True)

        prediction_df['predicted_winner'] = prediction_df.predicted_points.map(lambda x: 1 if x == prediction_df.predicted_points.max() else 0)
        prediction_df['actual_winner'] = prediction_df.actual_points.map(lambda x: 1 if x == prediction_df.actual_points.max() else 0)

        points += list(prediction_df.predicted_points.values)
        winners += list(prediction_df.predicted_winner.values)

        score += precision_score(prediction_df.actual_winner.values, prediction_df.predicted_winner.values)

        # Retrain model
        X_train = pd.concat([X_train, X_test])
        y_train = np.append(y_train, y_test)

        model.fit(X_train, y_train)
        

    return points, winners, score / df[df.season == N]['round'].nunique()

In [503]:
# parameters = parameters.sort_values('score', ascending=False)
parameters.head(5)

,model,criterion,max_features,max_depth,score
0,random_forest_regressor,squared_error,0.8,39,0.684211
1,random_forest_regressor,squared_error,0.8,11,0.684211
2,random_forest_regressor,squared_error,0.8,15,0.684211
3,random_forest_regressor,squared_error,0.8,55,0.684211
4,random_forest_regressor,squared_error,0.8,21,0.684211


In [504]:
params = parameters.iloc[0]
criterion = params.criterion
max_features = params.max_features	
max_depth = params.max_depth

params

model           random_forest_regressor
criterion                 squared_error
max_features                        0.8
max_depth                            39
score                          0.684211
Name: 0, dtype: object

In [505]:
# Random Forest Regressor

test = data.copy()[data.season == N]
test = test[['season', 'round', 'driver', 'starting_grid', 'podium', 'driver_points_from']].query('season ==@N')

model_params = (criterion, max_features, max_depth)
model = RandomForestRegressor(criterion=criterion, max_features=max_features, max_depth=max_depth)
model.fit(X_train, y_train)

points, winner = get_predictions(X_train, y_train, model)

test['predicted_points'] = points
test['predicted_winner'] = winner

In [507]:
rounds = len(test['round'].unique())
correct = len(test.query('predicted_winner == 1 & podium == 1'))

(correct / rounds) * 100

68.42105263157895

In [508]:
test.query('predicted_winner == 1')

,season,round,driver,starting_grid,podium,driver_points_from,predicted_points,predicted_winner
2764,2021,1,lewis_hamilton,2,1,25.0,18.548195,1
2785,2021,2,lewis_hamilton,1,2,19.0,21.701333,1
2804,2021,3,lewis_hamilton,2,1,25.0,18.260000,1
2824,2021,4,lewis_hamilton,1,1,25.0,19.510000,1
2844,2021,5,max_verstappen,2,1,25.0,17.590000,1
2877,2021,6,lewis_hamilton,2,15,0.0,17.440000,1
2883,2021,7,max_verstappen,1,1,26.0,22.930000,1
2902,2021,8,max_verstappen,1,1,25.0,17.630000,1
2922,2021,9,max_verstappen,1,1,26.0,20.030000,1
2943,2021,11,lewis_hamilton,1,2,18.0,14.850000,1
